Log in & import libraries

In [1]:
#connect to huggingface
from huggingface_hub import login
login()
#key:####

In [2]:
#from wandb import login
!wandb login #####

wandb: Appending key for api.wandb.ai to your netrc file: /home/et/.netrc


In [3]:
# import required libraries
import torch
from accelerate import Accelerator
from dataclasses import dataclass
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, PromptTuningConfig, get_peft_model, prepare_model_for_int8_training, set_peft_model_state_dict, TaskType, PromptTuningInit, PrefixTuningConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, set_peft_model_state_dict, PeftModel
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed
)
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl, default_data_collator
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

Set up Model for train

In [4]:
model_id = "bigcode/starcoder"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

/home/et/miniconda3/envs/starcoder_moe/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [6]:
# running training: train_dataset, valid_dataset
model = AutoModelForCausalLM.from_pretrained(
        "bigcode/starcoder",
        use_auth_token=True,
        use_cache=True,
        torch_dtype=torch.float16,
        device_map='auto',
)

/home/et/miniconda3/envs/starcoder_moe/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at bigcode/starcoder and are newly initialized: ['transformer.h.32.mlp.lora_A_c_proj_MLP.4.weight', 'transformer.h.22.mlp.lora_A_c_proj_MLP.4.weight', 'transformer.h.18.attn.lora_B_c_proj.4.weight', 'transformer.h.0.attn.lora_B_c_proj.4.weight', 'transformer.h.8.attn.lora_A_c_proj.1.weight', 'transformer.h.2.attn.lora_A_c_attn.0.weight', 'transformer.h.30.mlp.lora_B_c_proj_MLP.0.weight', 'transformer.h.32.attn.lora_B_c_proj.0.weight', 'transformer.h.25.attn.lora_A_c_proj.4.weight', 'transformer.h.22.attn.lora_B_c_proj.4.weight', 'transformer.h.26.attn.lora_A_c_attn.3.weight', 'transformer.h.10.mlp.lora_B_c_proj_MLP.4.weight', 'transformer.h.9.attn.lora_A_c_proj.2.weight', 'transformer.h.6.attn.lora_B_c_attn.4.weight', 'transformer.h.26.attn.lora_B_c_attn.2.weight', 'transformer.h.20.mlp.lora_B_c_proj_MLP.2.weight', 'transformer.h.6.attn.lora_A_c_proj.2.weight', 'transformer.h.1.attn.lora_B_c_attn.1.wei

In [7]:
# load the experts we saved before (one for each programming language)
EXPERTS_PATHS = ['models/LoRA_Experts/LoRA_cpp2py.pt', 'models/LoRA_Experts/LoRA_php2py.pt', 'models/LoRA_Experts/LoRA_js2py.pt', 'models/LoRA_Experts/LoRA_csharp2py.pt', 'models/LoRA_Experts/LoRA_java2py.pt']
model.load_experts(experts_paths=EXPERTS_PATHS)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Construct train/dev dataset

In [9]:
TRAIN_PATH = ####
VALID_PATH = ####
dataset = load_dataset('json', data_files={'train': TRAIN_PATH, 'dev': VALID_PATH},  use_auth_token=True, num_proc=None, streaming=True )

/home/et/miniconda3/envs/starcoder_moe/lib/python3.11/site-packages/datasets/load.py:2072: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


In [13]:
valid_data = dataset['dev']
train_data = dataset['train']
train_data = train_data.shuffle(buffer_size=5000, seed=128)

In [14]:
# calculate characters per token
def chars_token_ratio(dataset, tokenizer, input_column_name="prompt", output_column_name="completion", nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example, input_column_name, output_column_name)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [15]:
def prepare_sample_text(example, input_column_name="prompt", output_column_name="completion"):
    """Prepare the text from a sample of the dataset."""
    text = f"{example[input_column_name]} {example[output_column_name]}"
    return text

In [17]:
input_column_name = "code"
output_column_name = "py"
chars_per_token = chars_token_ratio(train_data, tokenizer, input_column_name, output_column_name)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

  0%|          | 0/400 [00:00<?, ?it/s]

The character to token ratio of the dataset is: 2.59


In [18]:
# find cap of 95% of training code len
len_list_train = [len(tokenizer.encode(prepare_sample_text(x, input_column_name='code', output_column_name='py'))) for x in train_data]
len_list_valid = [len(tokenizer.encode(prepare_sample_text(x, input_column_name='code', output_column_name='py'))) for x in valid_data]

In [19]:
def find_95th_percentile(len_list, p):
    len_list.sort()
    index = int(p* len(len_list))  # Index for the 95th percentile value
    percentile_value = len_list[index]
    return percentile_value

In [20]:
max_length_train = find_95th_percentile(len_list_train, .95)
max_length_valid = find_95th_percentile(len_list_valid, .95)
print(max_length_train)
print(max_length_valid)

999
1180


In [22]:
class TranslationDataset(IterableDataset):
    """
    Iterable dataset that returns constant length chunks of tokens from stream of text files.
        Args:
            tokenizer (Tokenizer): The processor used for proccessing the data.
            dataset (dataset.Dataset): Dataset with text files.
            max_length (int): The maximum length of tokens for each sample.
    """

    def __init__(
        self,
        tokenizer,
        dataset,
        max_length,
        input_column_name,
        output_column_name
    ):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.max_length=max_length
        self.concat_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else 49152 #default
        self.input_column_name = input_column_name
        self.output_column_name = output_column_name
        self.current_size = 0

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        buffer = []
        while more_examples:
            try:
                element = prepare_sample_text(next(iterator), self.input_column_name, self.output_column_name) + self.tokenizer.decode(self.tokenizer.eos_token_id)
                if (len(self.tokenizer.encode(element)) < (self.max_length+1)):
                    buffer.append(element)
            except StopIteration:
                more_examples = False
                break

        # The buffer is used to temporarily store tokenized examples, and buffer_len keeps track of the cumulative length of tokens in the buffer.
        tokenized_inputs = self.tokenizer(buffer, truncation=False, padding='max_length', max_length=self.max_length)["input_ids"]
        for tokenized_input in tokenized_inputs:
            self.current_size += 1
            yield {
                "input_ids": torch.LongTensor(tokenized_input),
                "labels": torch.LongTensor(tokenized_input)
                }

In [23]:
tokenizer.pad_token = tokenizer.eos_token
train_dataset = TranslationDataset(
        tokenizer=tokenizer,
        dataset=train_data,
        max_length=max_length_train,
        input_column_name='code',
        output_column_name='py'
)
valid_dataset = TranslationDataset(
        tokenizer=tokenizer,
        dataset=valid_data,
        max_length=max_length_valid,
        input_column_name='code',
        output_column_name='py'
)

Model training preparation

In [25]:
# calc the number of trainable parameters
for n,p in model.named_parameters():
    p.requires_grad = False
    if 'moe_gate' in n:
        p.requires_grad = True

print(f'There are {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters')

There are 30720 trainable parameters


In [26]:
# casting fp16 layers that will be trained to fp32
def prepare_model_fp16_for_training(model):
    for n,p in model.named_parameters():
        if p.requires_grad and p.dtype == torch.float16:
            p.data = p.data.float()

prepare_model_fp16_for_training(model)

In [27]:
args_output_dir = ####
args_max_steps = 1000
args_eval_freq_default = 50
args_log_freq_default = 50
args_save_freq_default = 1000
args_batch_size = 1 
args_learning_rate = 5e-5
args_lr_scheduler_type="cosine"
args_num_warmup_steps = 50
args_gradient_accumulation_steps_default = 4
args_weight_decay = 0.05

train_data.start_iteration = 0

training_args = TrainingArguments(
        output_dir=args_output_dir,
        evaluation_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        dataloader_drop_last=True,
        max_steps=args_max_steps,
        eval_steps=args_eval_freq_default,
        save_steps=args_save_freq_default,
        logging_steps=args_log_freq_default,
        per_device_train_batch_size=args_batch_size,
        per_device_eval_batch_size=args_batch_size,
        learning_rate=args_learning_rate,
        lr_scheduler_type=args_lr_scheduler_type,
        warmup_steps=args_num_warmup_steps,
        gradient_accumulation_steps=args_gradient_accumulation_steps_default,
        fp16=True,
        weight_decay=args_weight_decay,
        run_name=###, # eg. "moe_model"
        #push_to_hub=True,
)

In [28]:
class MyTrainer(Trainer):
    def log(self, logs) -> None:
        logs["learning_rate"] = self._get_learning_rate()
        super().log(logs)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    )

In [29]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joyce-pan (esperanto-tech). Use `wandb login --relogin` to force relogin


input_ids[0,1]:  tensor(1317, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  0.0018], device='cuda:0',
       grad_fn=<SliceBackward0>)
logits:  tensor([[0.1987, 0.2017, 0.1996, 0.2014, 0.1986]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
input_ids[0,1]:  tensor(3766, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  0.0018], device='cuda:0',
       grad_fn=<SliceBackward0>)
logits:  tensor([[0.1998, 0.1991, 0.2014, 0.2011, 0.1986]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
input_ids[0,1]:  tensor(1859, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  0.0018], device='cuda:0',
       grad_fn=<SliceBackward0>)
logits:  tensor([[0.1983, 0.1989, 0.2005, 0.2048, 0.1975]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
input_ids[0,1]:  tensor(754, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  

Step,Training Loss,Validation Loss,Rate
50,0.129300,0.121842,0.000050
100,0.133200,0.113530,0.000050
150,0.134600,0.111697,0.000049
200,0.133600,0.112682,0.000047
250,0.137800,0.111633,0.000045
300,0.132100,0.108336,0.000042
350,0.133500,0.107524,0.000039
400,0.131600,0.106527,0.000035
450,0.129800,0.106214,0.000031
500,0.133100,0.105508,0.000027


input_ids[0,1]:  tensor(3766, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  0.0018], device='cuda:0',
       grad_fn=<SliceBackward0>)
logits:  tensor([[0.2001, 0.1990, 0.1996, 0.2009, 0.2005]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
input_ids[0,1]:  tensor(3766, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  0.0018], device='cuda:0',
       grad_fn=<SliceBackward0>)
logits:  tensor([[0.1997, 0.1999, 0.1997, 0.1996, 0.2012]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
input_ids[0,1]:  tensor(3766, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,  0.0018], device='cuda:0',
       grad_fn=<SliceBackward0>)
logits:  tensor([[0.2005, 0.1988, 0.2011, 0.2004, 0.1992]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
input_ids[0,1]:  tensor(13320, device='cuda:0')
moe_weights sample:  tensor([-0.0037, -0.0117, -0.0047,  0.0060, -0.0084,

TrainOutput(global_step=1000, training_loss=0.13133525800704957, metrics={'train_runtime': 7684.3059, 'train_samples_per_second': 0.521, 'train_steps_per_second': 0.13, 'total_flos': 3.64665541459968e+17, 'train_loss': 0.13133525800704957, 'learning_rate': 0.0, 'epoch': 1.0})

In [31]:
# save the entire model to hub
trainer.push_to_hub()

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

'https://huggingface.co/jlpan/moe_test/tree/main/'

In [30]:
from collections import OrderedDict

SAVE_MoE_PATH = #### # Note: path is .pt file
moe_gate = OrderedDict({k:v for k,v in model.state_dict().items() if ('moe_gate' in k)})

torch.save(moe_gate, SAVE_MoE_PATH)

Multi-PL-to-Python translator evaluation

In [40]:
# Extract only python code from output
import re

def generate_clean_py_code(prompt, max_tokens=300, return_original_output=False):
    model.eval()
    prompt_tok = tokenizer(prompt, return_tensors='pt')

    with torch.no_grad():
        output = model.generate(input_ids=prompt_tok.input_ids.cuda(),
                                attention_mask=prompt_tok.attention_mask.cuda(),
                                max_new_tokens=max_tokens,
                                eos_token_id=tokenizer.eos_token_id,
                                )

    py_code = re.findall(r'<py>(.*?)'+tokenizer.eos_token, tokenizer.decode(output[0]))

    for p in py_code:
        if len(p) > 0:
            py_code = p
            break
    
    if return_original_output:
        return py_code, tokenizer.decode(output[0])
    return py_code

In [32]:
# load trained MoE gate
model.load_state_dict(torch.load(SAVE_MoE_PATH), strict=False)

_IncompatibleKeys(missing_keys=['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.attn.lora_A_c_attn.0.weight', 'transformer.h.0.attn.lora_A_c_attn.1.weight', 'transformer.h.0.attn.lora_A_c_attn.2.weight', 'transformer.h.0.attn.lora_A_c_attn.3.weight', 'transformer.h.0.attn.lora_A_c_attn.4.weight', 'transformer.h.0.attn.lora_B_c_attn.0.weight', 'transformer.h.0.attn.lora_B_c_attn.1.weight', 'transformer.h.0.attn.lora_B_c_attn.2.weight', 'transformer.h.0.attn.lora_B_c_attn.3.weight', 'transformer.h.0.attn.lora_B_c_attn.4.weight', 'transformer.h.0.attn.lora_A_c_proj.0.weight', 'transformer.h.0.attn.lora_A_c_proj.1.weight', 'transformer.h.0.attn.lora_A_c_proj.2.weight', 'transformer.h.0.attn.lora_A_c_proj.3.weight', 'transformer.h.0.attn.lora_A_c_proj.4.weight', 'tr

In [33]:
# eval PL to python

model.eval() 
device = 'cuda'

# change prompt_test to the PL to translate (with the form <code> PL_code <py>)
prompt_test ='<cpp> #include <bits/stdc++.h> NEW_LINE using namespace std ; void unefonctionlambda ( int matrice [ ] , int size ) { set < int > s ( matrice , matrice + size ) ; for ( auto x : s ) cout << x << \" \u2581 \" ; } int main ( ) { int matrice [ ] = { 1 , 3 , 2 , 2 , 1 } ; int n = sizeof ( matrice ) / sizeof ( matrice [ 0 ] ) ; unefonctionlambda ( arr , n ) ; return 0 ; } <py>'
input_ids = tokenizer(prompt_test, return_tensors='pt').to(device)
model.set_expert(input_ids)
output = model.generate(input_ids, max_new_tokens=tokenizer.encode(input_ids, return_tensors='pt').shape[1]+500) 
model.reset_expert()
tokenizer.decode(output[0], clean_up_tokenization_spaces=False)

Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')


Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using expert  tensor([0], device='cuda:0')
Using exper

'<cpp> #include <bits/stdc++.h> NEW_LINE using namespace std ; void unefonctionlambda ( int matrice [ ], int size ) { set < int > s ( matrice, matrice + size ) ; for ( auto x : s ) cout << x << " ▁ " ; } int main ( ) { int matrice [ ] = { 1, 3, 2, 2, 1 } ; int n = sizeof ( matrice ) / sizeof ( matrice [ 0 ] ) ; unefonctionlambda ( arr, n ) ; return 0 ; } <py> def unefonctionlambda ( matrice, size ) : NEW_LINE INDENT s = set ( matrice ) NEW_LINE for x in s : NEW_LINE INDENT print ( x, end = " ▁ " ) NEW_LINE DEDENT DEDENT if __name__ == " _ _ main _ _ " : NEW_LINE INDENT matrice = [ 1, 3, 2, 2, 1 ] NEW_LINE n = len ( matrice ) NEW_LINE unefonctionlambda ( matrice, n ) NEW_LINE DEDENT<|endoftext|>'